Get all the pre requisites for the tf-idf vectorization and the knn model building 

In [2]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np 

In [3]:
df=pd.read_csv('PreProcessedData.csv')

In [4]:
df.head()

,Unnamed: 0,name,ingredients,instructions,time,serving_size,tags,calories,protein,carbohydrates,fiber,fat,sodium
0,0,vaal ki usal recipe,"['two', 'sprout', 'vaal', 'field', 'bean', 'bu...","For vaal ki usal To make vaal ki usal recipe, ...",34 Mins,4 servings,"['nonstick', 'pan', 'boil', 'indian', 'recip',...",195.0,10.3,30.5,7.9,4.1,8.8
1,1,capsicum paneer sabzi recipe,"['two', 'zero', 'point', 'five', 'capsicum', '...",For capsicum paneer sabziTo make capsicum pane...,26 Mins,4 servings,"['non', 'stick', 'kadai', 'veg', 'antioxid', '...",74.0,2.6,9.5,3.2,2.9,21.8
2,2,restaurant style palak paneer recipe,"['one', 'slice', 'onion', 'three', 'roughli', ...",For the onion-cashew paste Combine all the ing...,46 Mins,4 servings,"['nonstick', 'pan', 'indian', 'dinner', 'india...",374.0,13.3,16.4,4.2,28.4,98.6
3,3,beetroot and dill salad recipe,"['one', 'oliv', 'oil', 'one', 'vinegar', 'must...",For beetroot and dill saladTo make beetroot an...,15 Mins,4 servings,"['refriger', 'indian', 'salad', 'indian', 'sal...",92.0,0.1,0.4,0.0,10.0,0.0
4,4,how to eat flaxseeds,"['one', 'flax', 'seed', 'one', 'lemon', 'juic'...",For flaxseeds mukhwas Combine all the ingredie...,5 Mins,4 servings,"['indian', 'dinner', 'indian', 'veg', 'recip',...",68.0,2.3,3.9,3.4,5.3,114.5


In [5]:
float_columns = ['calories', 'protein','carbohydrates','fiber','fat','sodium']  # Specify your float columns
for col in float_columns:
    df[col] = df[col].astype(str)  # Convert floats to strings
    # Optionally, handle NaN values before conversion
    df[col] = df[col].fillna('0')

In [6]:
df['combined']=df['ingredients']+" "+df['tags']

In [7]:
df['combined'][0]

"['two', 'sprout', 'vaal', 'field', 'bean', 'butter', 'bean', 'one', 'oil', 'one', 'cumin', 'seed', 'jeera', 'asafoetida', 'hing', 'five', 'six', 'curri', 'leav', 'kadi', 'patta', 'zero', 'point', 'five', 'fine', 'chop', 'onion', 'two', 'gingergarl', 'adraklehsun', 'past', 'zero', 'point', 'two', 'five', 'fine', 'chop', 'tomato', 'zero', 'point', 'five', 'turmer', 'powder', 'haldi', 'two', 'malvani', 'masala', 'five', 'kokum', 'zero', 'point', 'five', 'chop', 'jaggeri', 'gur', 'salt', 'zero', 'point', 'two', 'five', 'fine', 'chop', 'coriand', 'dhania'] ['nonstick', 'pan', 'boil', 'indian', 'recip', 'saut', 'indian', 'veg', 'recip', 'maharashtrian', 'recip', 'maharashtrian', 'dal', 'varan', 'amti', 'kalvan', 'proteinrichfoodsforpregn', 'pregnancydalsandveget', 'lowcaloriedalkadhisforweightloss', 'ironrichvegetablesd', 'lowcholesterolsubzisd', 'diabeticdalkadhirecip', 'highproteind', 'calciumrichdalkadhi']"

The TF/IDF vectorization on the relevant features

In [8]:
tfidf = TfidfVectorizer(
    stop_words='english',  # Remove common English stop words
    max_df=0.95,           # Ignore terms that appear in >80% of documents
    min_df=1,             # Adjust based on dataset size
    max_features=800,
    sublinear_tf=True   # Limit to top 1000 features
)


In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
X=df[['calories', 'protein','carbohydrates','fiber','fat','sodium']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled.shape


(3495, 6)

In [11]:
tfidf_matrix = tfidf.fit_transform(df['combined'])

In [12]:
tfidf_array=tfidf_matrix.toarray()
tfidf_array.shape

(3495, 800)

In [13]:
X_=np.concatenate([X_scaled,tfidf_array], axis=1)
X_.shape

(3495, 806)

Model building on our pre-processed ,vectorized dataset.

In [14]:
knn = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute')
knn.fit(X_)

NearestNeighbors(algorithm='brute', metric='cosine')

In [15]:
import pickle
with open('knn_model.pkl', 'wb') as f:
    pickle.dump(knn, f)

In [16]:
with open('tfidf.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

In [17]:
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)